# CSE 252B: Computer Vision II, Winter 2022 – Assignment 3
### Instructor: Ben Ochoa
### Due: Wednesday, February 16, 2022, 11:59 PM

## Instructions
* Review the academic integrity and collaboration policies on the course website.
* This assignment must be completed individually.
* All solutions must be written in this notebook
* Math problems must be done in Markdown/LATEX.
* You must show your work and describe your solution.
* Programming aspects of this assignment must be completed using Python in this notebook.
* Your code should be well written with sufficient comments to understand, but there is no need to write extra markdown to describe your solution if it is not explictly asked for.
* This notebook contains skeleton code, which should not be modified (This is important for standardization to facilate effecient grading).
* You may use python packages for basic linear algebra, but you may not use packages that directly solve the problem. If you are uncertain about using a specific package, then please ask the instructional staff whether or not it is allowable.
* You must submit this notebook exported as a pdf. You must also submit this notebook as an .ipynb file.
* Your code and results should remain inline in the pdf (Do not move your code to an appendix).
* You must submit both files (.pdf and .ipynb) on Gradescope. You must mark each problem on Gradescope in the pdf.
* It is highly recommended that you begin working on this assignment early.

## Problem 1 (Programming):  Estimation of the Camera Pose - Outlier rejection (20 points)
  Download input data from the course website.  The file
  hw3_points3D.txt contains the coordinates of 60 scene points
  in 3D (each line of the file gives the $\tilde{X}_i$, $\tilde{Y}_i$,
  and $\tilde{Z}_i$ inhomogeneous coordinates of a point).  The file
  hw3_points2D.txt contains the coordinates of the 60
  corresponding image points in 2D (each line of the file gives the
  $\tilde{x}_i$ and $\tilde{y}_i$ inhomogeneous coordinates of a
  point).  The corresponding 3D scene and 2D image points contain both
  inlier and outlier correspondences.  For the inlier correspondences,
  the scene points have been randomly generated and projected to image
  points under a camera projection matrix (i.e., $\boldsymbol{x}_i = \boldsymbol{P}
  \boldsymbol{X}_i$), then noise has been added to the image point
  coordinates.

  The camera calibration matrix was calculated for a $1280 \times 720$
  sensor and $45\,^\circ$ horizontal field of view lens.  The
  resulting camera calibration matrix is given by
  
  $\boldsymbol{K} = \left[
    \begin{array}{c c c}
      1545.0966799187809 & 0 & 639.5\\
      0 & 1545.0966799187809 & 359.5\\
      0 & 0 & 1
    \end{array}\right]$
    
  For each image point $\boldsymbol{x} = (x, y, w)^\top = (\tilde{x},
  \tilde{y}, 1)^\top$, calculate the point in normalized coordinates
  $\hat{\boldsymbol{x}} = \boldsymbol{K}^{-1} \boldsymbol{x}$.

  Determine the set of inlier point correspondences using the
  M-estimator Sample Consensus (MSAC) algorithm, where the maximum
  number of attempts to find a consensus set is determined adaptively.
  For each trial, use the 3-point algorithm of Finsterwalder (as
  described in the paper by Haralick et al.) to estimate the camera
  pose (i.e., the rotation $\boldsymbol{R}$ and translation $\boldsymbol{t}$ from the
  world coordinate frame to the camera coordinate frame), resulting in
  up to 4 solutions, and calculate the error and cost for each
  solution.  Note that the 3-point algorithm requires the 2D points in
  normalized coordinates, not in image coordinates.  Calculate the
  projection error, which is the (squared) distance between projected
  points (the points in 3D projected under the normalized camera
  projection matrix $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$) and the
  measured points in normalized coordinates (hint: the error tolerance
  is simpler to calculate in image coordinates using $\boldsymbol{P} =
  \boldsymbol{K} [\boldsymbol{R} | \boldsymbol{t}]$ than in normalized coordinates using
  $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$. You can avoid doing covariance propagation).

  
  Hint: this problem has codimension 2. 
 

#### Report your values for:
 * the probability $p$ that as least one of the random samples does not contain any outliers
 * the probability $\alpha$ that a given point is an inlier
 * the resulting number of inliers
 * the number of attempts to find the consensus set

In [1]:
import numpy as np
import time

def Homogenize(x):
    # converts points from inhomogeneous to homogeneous coordinates
    return np.vstack((x,np.ones((1,x.shape[1]))))

def Dehomogenize(x):
    # converts points from homogeneous to inhomogeneous coordinates
    return x[:-1]/x[-1]

def Normalize(K, x):
    # map the 2D points in pixel coordinates to the 2D points in normalized coordinates
    # Inputs:
    #   K - camera calibration matrix
    #   x - 2D points in pixel coordinates
    # Output:
    #   pts - 2D points in normalized coordinates
    
    """your code here"""
    pts = np.linalg.inv(K) @ Homogenize(x)
    return pts

    
# load data
x0=np.loadtxt('hw3_points2D.txt').T
X0=np.loadtxt('hw3_points3D.txt').T
print('x is', x0.shape)
print('X is', X0.shape)

K = np.array([[1545.0966799187809, 0, 639.5], 
      [0, 1545.0966799187809, 359.5], 
      [0, 0, 1]])

print('K =')
print(K)
    
def ComputeCost(P, x, X, K):
    # Inputs:
    #    P - normalized camera projection matrix
    #    x - 2D groundtruth image points
    #    X - 3D groundtruth scene points
    #    K - camera calibration matrix
    #
    # Output:
    #    cost - total projection error
    
    """your code here"""
    X = Homogenize(X)
    x_hat = K @ P @ X
    x_hat = Dehomogenize(x_hat)
    epsilon = np.sum(np.power((x - x_hat), 2), axis = 0)
    cost = np.sum(epsilon)
    return cost



x is (2, 60)
X is (3, 60)
K =
[[1.54509668e+03 0.00000000e+00 6.39500000e+02]
 [0.00000000e+00 1.54509668e+03 3.59500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [11]:
from scipy.stats import chi2
import random

class FinsterWalderSolution(object):
    def __init__(self, selected_2D_pts, selected_3D_pts, K):
        self.pts = Normalize(K, selected_2D_pts)
        self.X = selected_3D_pts
        
    def convertToUnitVector(self, d):
        # d: 3x1 vector
        d = d / (np.sign(d[2]) * np.linalg.norm(d))
        return d.reshape((3,1))
    
    def calculateDistance(self, pt1, pt2):
        dist = np.linalg.norm(pt1-pt2)
        return dist**2
    
    def calculateSinCos(self, d1, d2):
        cosD = d1.T.dot(d2)
        return 1 - (cosD**2), cosD
    
    def calculateUV(self):
        d1 = self.convertToUnitVector(self.pts[:,0])
        d2 = self.convertToUnitVector(self.pts[:,1])
        d3 = self.convertToUnitVector(self.pts[:,2])
        
        a = self.calculateDistance(self.X[:, 1], self.X[:, 2])
        b = self.calculateDistance(self.X[:, 0], self.X[:, 2])
        c = self.calculateDistance(self.X[:, 0], self.X[:, 1])
        
        sin2A, cosA = self.calculateSinCos(d2, d3)
        sin2B, cosB = self.calculateSinCos(d1, d3)
        sin2Y, cosY = self.calculateSinCos(d1, d2)
        
        G = c * ((c * sin2B) - (b * sin2Y))
        H = (b * (b - a)* sin2Y) + (c * (c + 2*a)*sin2B) + (2 * b * c * (cosA*cosB*cosY - 1))
        I = (b * (b - c)* sin2A) + (a * (a + 2*c)*sin2B) + (2 * a * b * (cosA*cosB*cosY - 1))
        J = a * ((a * sin2B) - (b * sin2A))
        
        lam = np.roots(np.array([G, H, I, J]).reshape(-1))
        lambdas = np.real(lam[np.isreal(lam)])
        
        uvs_lst = []
        lam = lambdas[random.sample(range(len(lambdas)), 1)] # taking any of the real roots

        A = b * (lam + 1)
        B = -b * cosA
        C = ( b - a - lam * c )
        D = -b * lam * cosY
        E = cosB * ((a + lam * c))
        F = ( -a  + lam * (b - c) )
        
        p = (B**2) - A*C
        q = (E**2) - C*F
        if(p < 0 or q < 0):
            return uvs_lst, d1, d2, d3
        
        p = np.sqrt(p)
        q = np.sign(B*E - C*D) * np.sqrt(q)
        M1 = (-B + p) / C
        N1 = -(E - q) / C
        M2 = (-B - p) / C
        N2 = -(E + q) / C
        MN = np.array([[M1, N1],[M2,N2]])
        for m, n in MN:
            A = b - (m**2) * c
            B = 2 * ((c * (cosB - n) * m) - (b * cosY))
            C = -c * (n**2) + (2 * c * n) * cosB + b - c
                
            eq_sol = np.roots(np.array([A, B, C]).reshape(-1))
            eq_sol = np.real(eq_sol[np.isreal(eq_sol)])
            for sol in eq_sol:
                v = sol * m + n
                s1 = np.sqrt(b / (1 + (v**2) - (2*v*cosB)))
                s2 = sol * s1
                s3 = v * s1
                if(s1 > 0 and s2 > 0 and s3 > 0):
                    uvs_lst.append([s1[0,0], s2[0,0], s3[0,0]])                
        return uvs_lst, d1, d2, d3
    
    def calcPoints(self):
        ptsList = []
        uvs_lst, d1, d2, d3 = self.calculateUV()
        for s1, s2, s3 in uvs_lst:
            X1 = s1 * d1
            X2 = s2 * d2
            X3 = s3 * d3
            ptsList.append(np.hstack((X1, X2, X3)))
        return ptsList
    
def calculateModel(cam_coords, world_coords):
    '''
        Helper function to calculate 3D euclidean transformation
    '''
    cam_mean = np.mean(cam_coords, axis = 1).reshape(3, 1) # 3x1
    world_mean = np.mean(world_coords, axis = 1).reshape(3, 1)
    C = cam_coords.T - cam_mean.T
    B = world_coords.T - world_mean.T
    S = np.dot(C.T, B)
    U, sigma, V = np.linalg.svd(S)
    
    if(np.linalg.det(U)*np.linalg.det(V.T) < 0):
        I = np.eye(3)
        I[-1, -1] = -1
        R = U @ I @ V # 3x3
    else:
        R = U @ V
        
    T = (cam_mean - np.dot(R, world_mean)).reshape((R.shape[0],-1)) # 3x1
    return np.hstack((R, T)) # [R\t], 3x4 matrix
    
def calculateError(x, X, K, P):
    '''
        Helper function to calculate reprojection error
    '''
    x_estimate = Dehomogenize(K @ np.matmul(P, Homogenize(X)))
    epsilon = x - x_estimate
    error = np.sum(epsilon**2, axis=0)
    return error

def calculateConsensusCost(error, tol, count):
    '''
        Helper function to calculate consensus cost
    '''
    inlier_indices = []
    cost = 0
    for i in range(count):
        if(error[i] <= tol):
            cost += error[i]
            inlier_indices.append(i)
        else:
            cost += tol
    return cost, inlier_indices
    
    
def MSAC(x, X, K, thresh, tol, p):
    # Inputs:
    #    x - 2D inhomogeneous image points
    #    X - 3D inhomogeneous scene points
    #    K - camera calibration matrix
    #    thresh - cost threshold
    #    tol - reprojection error tolerance 
    #    p - probability that as least one of the random samples does not contain any outliers   
    #
    # Output:
    #    consensus_min_cost - final cost from MSAC
    #    consensus_min_cost_model - camera projection matrix P
    #    inliers - list of indices of the inliers corresponding to input data
    #    trials - number of attempts taken to find consensus set
    
    """your code here"""
    trials = 0
    max_trials = np.inf
    consensus_min_cost = np.inf
    consensus_min_cost_model = np.zeros((3,4))
    inliers = np.random.randint(0, 59, size=10)
    
    while(trials < max_trials and consensus_min_cost > thresh):
        indices = random.sample(range(x.shape[1]), 3)
        x_points = x[:, indices]
        X_points = X[:, indices]
        
        fw = FinsterWalderSolution(x_points, X_points, K)
        camera_points = fw.calcPoints()
        for pt in camera_points:
            model = calculateModel(pt, X_points)
            error = calculateError(x, X, K, model)
            consensus_cost, inlier_indices = calculateConsensusCost(error, tol, x.shape[1])
            if(consensus_cost < consensus_min_cost):
                consensus_min_cost = consensus_cost
                consensus_min_cost_model = model
                
                inliers = inlier_indices
                w = len(inlier_indices) / x.shape[1]
                max_trials = np.log(1 - p) / np.log(1 - (w**3))
            
        trials += 1
        
    
    return consensus_min_cost, consensus_min_cost_model, inliers, trials


# MSAC parameters 
thresh = 100
codim = 2
p = 0.99
alpha = 0.95
tol = chi2.ppf(alpha, codim) * 1 # inverse chi squared cdf is ppf, variance = 1

tic=time.time()

cost_MSAC, P_MSAC, inliers, trials = MSAC(x0, X0, K, thresh, tol, p)

# choose just the inliers
x = x0[:,inliers]
X = X0[:,inliers]

toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
print('%d iterations'%trials)
print('inlier count: ',len(inliers))
print('MSAC Cost=%.9f'%cost_MSAC)
print('P = ')
print(P_MSAC)
print('inliers: ',inliers)


took 0.024258 secs
15 iterations
inlier count:  43
MSAC Cost=187.011474774
P = 
[[  0.27523884  -0.69520392   0.66402943   5.18518884]
 [  0.66045319  -0.36518151  -0.65608235   7.46419433]
 [  0.69860229   0.6191397    0.35863751 175.94171836]]
inliers:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 48, 49]


#### Final values for parameters
* $p$ = 0.99
* $\alpha$ = 0.95
* tolerance = 5.991464547107979
* num_inliers = 46
* num_attempts = 10

## Problem 2 (Programming): Estimation of the Camera Pose - Linear Estimate (30 points)
  Estimate the normalized camera projection matrix
  $\hat{\boldsymbol{P}}_\text{linear} = [\boldsymbol{R}_\text{linear} |
  \boldsymbol{t}_\text{linear}]$ from the resulting set of inlier
  correspondences using the linear estimation method (based on the
  EPnP method) described in lecture. Report the resulting 
  $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$.

In [3]:
def EPnP(x, X, K):
    # Inputs:
    #    x - 2D inlier points
    #    X - 3D inlier points
    # Output:
    #    P - normalized camera projection matrix
    
    """your code here"""
    mean_3D = np.mean(X, axis=1)[:,None]
    cov_sq = np.cov(X)
    U_X, D_X, V = np.linalg.svd(cov_sq)
    sigma_sq = np.sum(D_X)
    
    C_1 = mean_3D
    
    alphas = V @ (X - C_1)
    alpha_2 = alphas[0,:].reshape(1, alphas.shape[1])
    alpha_3 = alphas[1,:].reshape(1, alphas.shape[1])
    alpha_4 = alphas[2,:].reshape(1, alphas.shape[1])
    alpha_1 = (1 - alpha_2 - alpha_3 - alpha_4).reshape(1, alphas.shape[1])
    alpha = np.vstack((alpha_1, alpha_2, alpha_3, alpha_4))
    
    x_hat = Dehomogenize(np.linalg.inv(K) @ Homogenize(x))
    M = np.zeros((2*x.shape[1],12))
    for i in range(x.shape[1]):
        xi = x_hat[0,i]
        yi = x_hat[1,i]
        alphai1 = alpha_1[0,i]
        alphai2 = alpha_2[0,i]
        alphai3 = alpha_3[0,i]
        alphai4 = alpha_4[0,i]
        
        M[2*i: 2*i+2,:] = np.array([[alphai1, 0, -alphai1*xi, alphai2, 0, -alphai2*xi, alphai3, 0, -alphai3*xi, alphai4, 0, -alphai4*xi],
                                    [0, alphai1, -alphai1*yi, 0, alphai2, -alphai2*yi, 0, alphai3, -alphai3*yi, 0, alphai4, -alphai4*yi]])

    U, Sigma, V = np.linalg.svd(M)
    C = V[-1, :]
    C_1 = C[0:3].reshape(3,1)
    C_2 = C[3:6].reshape(3,1)
    C_3 = C[6:9].reshape(3,1)
    C_4 = C[9:].reshape(3,1)
    
    X_Cam = np.hstack((C_1, C_2, C_3, C_4)) @ alpha
    
    
    mean_cam = np.mean(X_Cam, axis = 1)
    cov_cam = np.cov(X_Cam)
    sigma_cam = np.trace(cov_cam)
    beta = np.sqrt(sigma_sq / sigma_cam)
    if(np.sign(mean_cam[-1]) < 0):
        beta = -beta
    X_Cam = beta * X_Cam
    P = calculateModel(X_Cam, X)
    return P

# Uncomment the following block to run EPnP on an example set of inliers.
# You MUST comment it before submitting, or you will lose points.
# The sample cost with these inliers should be around 57.82.
# inliers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
# x = x0[:,inliers]
# X = X0[:,inliers]

tic=time.time()
P_linear = EPnP(x, X, K)
toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
print('R_linear = ')
print(P_linear[:,0:3])
print('t_linear = ')
print(P_linear[:,-1])

took 0.001822 secs
R_linear = 
[[ 0.27621732 -0.69327622  0.6656366 ]
 [ 0.66520522 -0.36197974 -0.65304876]
 [ 0.69369014  0.62316831  0.36118589]]
t_linear = 
[  5.31626974   7.99694248 175.81540474]


## Problem 3 (Programming): Estimation of the Camera Pose - Nonlinear Estimate (30 points) 
  Use $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$ as an
  initial estimate to an iterative estimation method, specifically the
  Levenberg-Marquardt algorithm, to determine the Maximum Likelihood
  estimate of the camera pose that minimizes the projection error
  under the normalized camera projection matrix $\hat{\boldsymbol{P}} =
  [\boldsymbol{R} | \boldsymbol{t}]$.  You must parameterize the camera rotation
  using the angle-axis representation $\boldsymbol{\omega}$ (where
  $[\boldsymbol{\omega}]_\times = \ln \boldsymbol{R}$) of a 3D rotation, which is
  a 3-vector.
  
  
  Report the initial cost (i.e. cost at iteration 0) and the cost at the end
  of each successive iteration. Show the numerical values for the final 
  estimate of the camera rotation $\boldsymbol{\omega}_\text{LM}$ and 
  $\boldsymbol{R}_\text{LM}$, and the camera translation $\boldsymbol{t}_\text{LM}$.

In [4]:
from scipy.linalg import block_diag

# Note that np.sinc is different than defined in class
def Sinc(x):
    """your code here"""
    if(x == 0):
        return 1
    return np.sin(x) / x

def dSinc(x):
    if(x == 0):
        return 0
    else:
        val = (np.cos(x) / x) - (np.sin(x) / np.power(x, 2))
        return val

def skew(w):
    # Returns the skew-symmetrix represenation of a vector
    """your code here"""
    w = w.ravel()
    w_skew = np.array([
        [0, -w[2], w[1]],
        [w[2], 0, -w[0]],
        [-w[1], w[0], 0]
    ])
    return w_skew


def Parameterize(R):
    # Parameterizes rotation matrix into its axis-angle representation
    """your code here"""
    _, _, Vt = np.linalg.svd(R - np.eye(3))
    V = Vt[-1, :] # V is the last row of Vt
    V_hat = np.array([R[2,1] - R[1,2], R[0,2] - R[2,0], R[1,0] - R[0,1]]).T
    sinTheta = (V_hat.T.dot(V)) / 2
    cosTheta = (np.trace(R) - 1) / 2
    
    theta = np.arctan2(sinTheta, cosTheta)
    
    if(-1e-5 < theta < 1e-5): # for small rotations
        w = 0.5 * V_hat
        return w, theta
    
    w = theta * V
    if(np.linalg.norm(w) > np.pi):
        w_norm = np.linalg.norm(w)
        w = w * (1 - (2*np.pi / w_norm) * np.ceil((w_norm - np.pi) / (2*np.pi)))
    return w, theta


def Deparameterize(w):
    # Deparameterizes to get rotation matrix
    """your code here"""
    theta = np.linalg.norm(w)
    w_skew = skew(w)
    
    if(-1e-5 < theta < 1e-5): # for small rotations
        R = np.eye(3) + w_skew
        return R
    
    R = (np.cos(theta) * np.eye(3)) + (Sinc(theta) * w_skew) + ((1 - np.cos(theta)) / (theta**2)) * np.dot(w, w.T)
    return R


def DataNormalize(pts):
    # Input:
    #    pts - 3D scene points
    # Outputs:
    #    pts - data normalized points
    #    T - corresponding transformation matrix
    
    """your code here"""
    num_axes = pts.shape[0]
    var = np.var(pts, axis=1)
    mean = np.mean(pts, axis=1)
    var_sum = np.sum(var)
    s = np.sqrt(3 / var_sum)
        
    T = np.eye(pts.shape[0]+1)
    
    diagonal = np.eye(pts.shape[0]) * s
    T[:num_axes, :num_axes] = diagonal
    
    T[:num_axes, -1] = -mean * s
    pts = np.matmul(T, Homogenize(pts))
    return pts, T

    
def Normalize_withCov(K, x, covarx):
    # Inputs:
    #    K - camera calibration matrix 
    #    x - 2D points in pixel coordinates
    #    covarx - covariance matrix
    #
    # Outputs:
    #    pts - 2D points in normalized coordinates
    #    covarx - normalized covariance matrix
    
    """your code here"""
    K_inv = np.linalg.inv(K)
    J_inv = K_inv[:2,:2]
    pts = Normalize(K, x)
    for i in range(x.shape[1]):
        covarx[2*i: 2*i+2, 2*i: 2*i+2] = J_inv @ covarx[2*i: 2*i+2, 2*i: 2*i+2] @ J_inv.T
    return pts, covarx
    

def Jacobian(R, w, t, X):
    # compute the jacobian matrix
    # Inputs:
    #    R - 3x3 rotation matrix
    #    w - 3x1 axis-angle parameterization of R
    #    t - 3x1 translation vector
    #    X - 3D inlier points
    #
    # Output:
    #    J - Jacobian matrix of size 2*nx6
    
    """your code here"""
    X = Dehomogenize(X)
    X_rotated = np.dot(R, X)
    x_hat = Dehomogenize(X_rotated + t)
    J = np.zeros((2*X.shape[1],6))
    for i in range(X.shape[1]):        
        w_hat = X_rotated[-1,i] + t[-1,0]
        dx_dXrot = dx_dt = np.array([[1/w_hat, 0, -x_hat[0,i] / w_hat],
                                     [0, 1/w_hat, -x_hat[1,i] / w_hat]])
        
        theta = np.linalg.norm(w)
        Xi = X[:, i].reshape(3,1)
        if(-1e-5 < theta < 1e-5): # for small rotations
            dXrot_dw = skew(-Xi)
        else:
            s = (1 - np.cos(theta)) / (theta**2)
            ds_dtheta = (theta * np.sin(theta) - 2*(1 - np.cos(theta))) / (theta**3)
            skew_x = skew(-Xi)
            skew_w = skew(w)
            skew_wx = skew(-np.cross(w.ravel(), Xi.ravel()).reshape(3,1))
            dtheta_dw = 1 / theta * w.T
            dXrot_dw = (Sinc(theta) * skew_x) + \
                       ((np.cross(w.ravel(), Xi.ravel()).reshape(3,1)) * dSinc(theta) * dtheta_dw) + \
                       (np.cross(w.ravel(), np.cross(w.ravel(), Xi.ravel())).reshape(3,1) * ds_dtheta * dtheta_dw) + \
                       (s * (skew_w.dot(skew_x) + skew_wx))
            
        dx_dw = np.dot(dx_dXrot, dXrot_dw)
        J[2*i : 2*i+2, :] = np.hstack((dx_dw, dx_dt))
    return J


def ComputeCost_withCov(P, x, X, covarx):
    # Inputs:
    #    P - normalized camera projection matrix
    #    x - 2D ground truth image points in normalized coordinates
    #    X - 3D groundtruth scene points
    #    covarx - covariance matrix
    #
    # Output:
    #    cost - total projection error

    """your code here"""
    epsilon = (Dehomogenize(x) - Dehomogenize(np.dot(P, X))).reshape((2*X.shape[1], 1), order='F')
    cost = np.sum(epsilon.T @ np.linalg.inv(covarx) @ epsilon)
    return cost


In [5]:
def LM(P, x, X, K, max_iters, lam):
    # Inputs:
    #    P - initial estimate of camera pose
    #    x - 2D inliers
    #    X - 3D inliers
    #    K - camera calibration matrix 
    #    max_iters - maximum number of iterations
    #    lam - lambda parameter
    #
    # Output:
    #    P - Final camera pose obtained after convergence
    
    covarx = np.eye(2*X.shape[1])
    """your code here"""
    X, U = DataNormalize(X)
    x, covarx = Normalize_withCov(K, x, covarx)
    
    R = P[:, 0:3]
    t = P[:,-1].reshape(3, 1)
        
    cam_center = (-R.T @ t).reshape(3, 1)
    cam_center_norm = U @ Homogenize(cam_center)
    t = -R @ Dehomogenize(cam_center_norm)
    P = np.hstack((R, t))
    
    i = 0
    cost_prev = ComputeCost_withCov(P, x, X, covarx)
    print("Initial cost %.9f"%(cost_prev))
    while i < max_iters:
        R = P[:, 0:3]
        w, theta = Parameterize(R)
        
        J = Jacobian(R, w, t, X)
        
        eps = (Dehomogenize(x) - Dehomogenize(np.dot(P, X))).reshape((2*X.shape[1], 1), order='F')
        A = J.T @ np.linalg.inv(covarx) @ J
        b = J.T @ np.linalg.inv(covarx) @ eps
        
        # inner loop
        A_augmented = A + lam * np.eye(J.shape[1])
        delta = np.linalg.inv(A_augmented) @ b
        w_0 = (w + delta[0:3,0]).reshape(3,1)
        t_0 = (t.ravel() + delta[3:6,0]).reshape(3,1)
        R_0 = Deparameterize(w_0)
        P_0 = np.hstack((R_0, t_0))
        eps_0 = (Dehomogenize(x) - Dehomogenize(np.dot(P_0, X))).reshape((2*X.shape[1], 1), order='F')
        cost_0 = ComputeCost_withCov(P_0, x, X, covarx)
        
        if(1 - cost_0/cost_prev < 1e-15):
            break
        
        if(cost_0 < cost_prev):
            P = P_0
            eps = eps_0
            lam = 0.1 * lam
            cost_prev = cost_0
            print ('iter %03d Cost %.9f'%(i+1, cost_0))
            i = i + 1
        else:
            lam = 10.0 * lam
    
    cam_center_norm = (-P[:, 0:3].T @ P[:,-1]).reshape(3,1)
    cam_center = np.linalg.inv(U) @ Homogenize(cam_center_norm)
    t = -P[:, 0:3] @ Dehomogenize(cam_center)
    P = np.hstack((P[:, 0:3], t))
    return P
 
# With the sample inliers...
# Start: 57.854356308
# End: 57.815478730

# LM hyperparameters
lam = .001
max_iters = 100

tic = time.time()
P_LM = LM(P_linear, x, X, K, max_iters, lam)
w_LM,_ = Parameterize(P_LM[:,0:3])
toc = time.time()
time_total = toc-tic

# display the results
print('took %f secs'%time_total)
print('w_LM = ')
print(w_LM)
print('R_LM = ')
print(P_LM[:,0:3])
print('t_LM = ')
print(P_LM[:,-1])
# np.isclose for for w, 
      

Initial cost 40.849447092
iter 001 Cost 40.784220511
took 0.022825 secs
w_LM = 
[ 1.32841476 -0.02913122  1.41510412]
R_LM = 
[[ 0.27589328 -0.6935649   0.66547023]
 [ 0.66559028 -0.3616362  -0.65284672]
 [ 0.69344969  0.62304654  0.36185706]]
t_LM = 
[  5.28222329   8.03889647 175.80418342]
